## DS 7337: Natural Language Processing

## Jaclyn Coate
## Homework 5

#### Spring 2021
#### Natural Language Processing w/ Python: Bird, Klein, & Loper

In [1]:
# Environment Prep
import bs4; print( 'bs4 ' + bs4.__version__)
from bs4 import BeautifulSoup, SoupStrainer

import nltk; print( 'nltk ' + nltk.__version__)
from nltk import word_tokenize, pos_tag, RegexpParser;
from nltk.tokenize import sent_tokenize

import re; print('re ' + re.__version__)
import requests; print('requests ' + requests.__version__)

bs4 4.8.2
nltk 3.4.5
re 2.2.1
requests 2.22.0


##### HW 5: Question 1
Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8, which together will make up your semester project.   

a. It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.  

b. Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.  

c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.  

d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.  

In [16]:
# Source links for Top Horror Movies in History and store in review_home_url
review_home_url = {
    'The Exorcist': 'https://www.imdb.com/title/tt0070047/reviews?ref_=tt_ql_3',
    'Paranormal Activity': 'https://www.imdb.com/title/tt1179904/reviews?ref_=tt_ql_3',
    'Hellraiser': 'https://www.imdb.com/title/tt0093177/reviews?ref_=tt_ql_3',
    'A Nightmare on Elm Street': 'https://www.imdb.com/title/tt0087800/reviews?ref_=tt_ql_3',
    'American Psycho' : 'https://www.imdb.com/title/tt0087800/reviews?ref_=tt_ql_3',
    'The Blair Witch Project': 'https://www.imdb.com/title/tt0185937/reviews?ref_=tt_ql_3',
}

In [20]:
# This function is to grab text from the URL: grab_text_from_url
def get_text_from_url(url):
    return requests.get(url).text
text = get_text_from_url(review_home_url['The Exorcist'])

In [21]:
# This function points to the links within the HTML
def get_all_html_links(html):
    tags = BeautifulSoup(html, 'html.parser', parse_only=SoupStrainer('a', href=True))
    urls = [str(tag.attrs['href']) for tag in tags]
    return urls
all_links = get_all_html_links(text)

In [19]:
# This defintion sets the URL template and allows the unique urls to be grabbed after the root url
def get_review_links(links):
    url_template = 'https://www.imdb.com{}'
   
    return [url_template.format(link) for link in links]

urls = get_review_links(all_links);

NameError: name 'get_review_links' is not defined

In [ ]:
# This defines the relevent part of the link by calling our review section to point to HTML post the root 'review'
def relevent_link(link):
    if '/review/' in link:
        return True
    return False
def get_relevent_links(links):
    relevent_links = filter(relevent_link, all_links)
    unique_relevent_links = set(relevent_links)
    return list(unique_relevent_links)
relevent_urls = get_relevent_links(urls)
len(relevent_urls)

##### HW 5: Question 2
Extract noun phrase (NP) chunks from your reviews using the following procedure:  

a. In Python, use BeautifulSoup to grab the main review text from each link.  

In [4]:
# Grabbing reviews rfom sites
def strain_content(name, attrs):
    if name == 'div' and dict(attrs).get('class', None) == 'content':
        return True
    return False
def clean_review_text(text):
    return re.split('\\n\\n\s+\d+ out of \d+', text)[0]
def get_review_from_url(url):
    html = get_text_from_url(url)
    tags = BeautifulSoup(html, 'html.parser', parse_only=SoupStrainer(strain_content))
    review = clean_review_text(tags.text)
    return review

In [5]:
# Grabbing reviews rfom sites
def get_review_from_site(url):
    reviews = []

    reviews_home_text = get_text_from_url(url)
    all_links = get_all_html_links(reviews_home_text)
    relevent_links = get_relevent_links(all_links)

    review_urls = get_review_urls_from_links(relevent_links)
    for url in review_urls:
        reviews.append(get_review_from_url(url))
        # break
    return reviews

In [6]:
# Locating the review sectiosn from the sites 
def get_reviews_from_all_sites():
    all_reviews = []
    review_titles = review_home_urls.keys()
    for title in review_titles:
        review_home_url = review_home_urls[title]
        all_reviews = all_reviews + get_review_from_site(review_home_url)
    return all_reviews

In [7]:
# The defintions for Part of Speech tagging
def pos_tagging_for_review(sentence):
    cleaned_review = sentence.lower()
    tokenized_review = word_tokenize(cleaned_review)
    return pos_tag(tokenized_review)
def get_chunking_for_sentence(sentence):
    tagged_sentence = pos_tagging_for_review(sentence)

    grammar = "NP: {<DT>?<JJ>*<NN>}"

    cp = nltk.RegexpParser(grammar)
    result = cp.parse(tagged_sentence)
    return result

def get_chunking_for_review(review):
    sentences = sent_tokenize(review)
    result = [get_chunking_for_sentence(sentence) for sentence in sentences]
    return result

b. Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser. 

In [8]:
# This is the funtion for chunking the reviews
def get_chunking_for_reviews(reviews):
    return [get_chunking_for_review(review) for review in reviews]

In [9]:
# Function for identifying noun phrases for chunking
def get_NP_for_review(review):
    main_trees = get_chunking_for_review(review)
    subtrees = []
    for main_tree in main_trees:
        for subtree in main_tree.subtrees():
            if subtree.label() == 'NP':
                subtrees.append(subtree)
    return subtrees[1:]

In [10]:
# Returning those noun phrases
def get_NP_for_reviews(reviews):
    return [get_NP_for_review(review) for review in reviews]

In [11]:
# Grabbign reviews and placing them in one place
all_reviews = get_reviews_from_all_sites()

NameError: name 'review_home_urls' is not defined

In [12]:
# Quick view of all reivews
print(len(all_reviews))

NameError: name 'all_reviews' is not defined

c. You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.  

In [13]:
noun_phrases_reviews = get_NP_for_reviews(all_reviews)

NameError: name 'all_reviews' is not defined

In [14]:
# Grabbing noun phrases w/ 4 word range
for noun_phrases in noun_phrases_reviews[0:4]:
    for i in range(4):
        print(noun_phrases[i])
    print()

NameError: name 'noun_phrases_reviews' is not defined

#### HW 5: Question 3

Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).

In order to collect movies and their reviewes I sourced reviews from the reviews pages of the top 6 horror movies of all time. I sourced the reviews from IMDB.com, one of the most proliferent movie review pages of all time. I collected and sourced the movies parsing HTML out of URL with the tool beautifulsoup in Python. I used NLTK to word chunk and identify *noun phrases* for review. 

In [15]:
noun_phrases_reviews

NameError: name 'noun_phrases_reviews' is not defined